In [ ]:
import bigframes.pandas as bpd
import pandas as pd
import numpy as np

data=pd.Series(np.random.randn(8),
               index=[["a","a","a","b",
                       "b","b","c","c"],
                      [1,2,3,1,2,3,1,2]])
data
bq_data = bpd.read_pandas(data)
print(bq_data)

In [ ]:
data.loc["a":"b"]

In [ ]:
bq_data.loc["a": "b"]

In [1]:
import bigframes.pandas as bpd

idx = bpd.Index(['Apple', 'Banana', 'Orange'])
print(idx.get_loc('Banana'))  # Output: 1 

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/core/global_session.py:103: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  _global_session = bigframes.session.connect(


1


In [ ]:
monotonic_index = bpd.Index(list('abbc'))
monotonic_index.get_loc('b') # Output: slice(1, 3, None)


In [2]:
non_monotonic_index = bpd.Index(list('abcb'))
non_monotonic_index.get_loc('b') # Expect array([False,  True, False,  True])

0    False
1     True
2    False
3     True
dtype: boolean

In [ ]:
import bigframes.pandas as bpd
import pandas as pd

bpd.options.bigquery.project = 'bigframes-dev'  # project #: 1084210331973
bpd.options.bigquery.location = "us"

@bpd.udf(
        dataset='jialuo_test_us', name='test',
        packages = ['pypdf[crypto]'],
        )
def func(s: bpd.Series) -> bool:
    return s['a'] + s['b'] > 0

bdf = {'a': [0, 1, 2], 'b': [3, 4, 5]}
res = bdf.apply(func, axis=1)
print(res)

res1 = bdf.where(func)

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/pandas/__init__.py:149: PreviewWarning: udf is in preview.
  return global_session.with_default_session(
/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/functions/_function_session.py:971: PreviewWarning: input_types=Series is in preview.
  warnings.warn(msg, stacklevel=1, category=bfe.PreviewWarning)
/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/functions/_utils.py:86: FunctionPackageVersionWarning: numpy, pandas, and pyarrow versions in the function execution
environment may not precisely match your local environment.
  warnings.warn(msg, category=bfe.FunctionPackageVersionWarning)
